# Sampling in CNID

In this tutorial, let us verify the computed CNID. 

We do this by sampling the total energy of a system of Si(111)/SiC(0001) interface with 
displacing one crystal in the CNID.

## Again, making an two-D CSL interface following the previous tutorials

In [1]:
from interfacemaster.cellcalc import get_primitive_hkl, get_pri_vec_inplane, get_normal_index
from interfacemaster.interface_generator import core, print_near_axis, convert_vector_index, write_trans_file
from numpy import array, dot, round, cross
from numpy.linalg import inv, det

In [2]:
my_interface = core('cif_files/Si_mp-149_conventional_standard.cif',\
                    'cif_files/SiC_mp-568656_conventional_standard.cif')

Warning!, this programme will rewrite the POSCAR file in this dir!


/Users/jasonxie/.local/lib/python3.8/site-packages/pymatgen/io/cif.py:1121: UserWarning: Issues encountered while parsing CIF: Some fractional co-ordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: %s" % "\n".join(self.warnings))


In [3]:
hkl_1 = get_primitive_hkl([1,1,1], my_interface.conv_lattice_1, my_interface.lattice_1)
hkl_2 = get_primitive_hkl([0,0,1], my_interface.conv_lattice_2, my_interface.lattice_2)

In [4]:
my_interface.parse_limit(du = 2e-2, S  = 2e-2, sgm1=50, sgm2=50, dd = 2e-2)

#Do searching!
my_interface.search_one_position_2D(hkl_1, hkl_2, theta_range = 180, \
                                    dtheta = 0.01, start = 15.21, integer_tol=1e-3)

2
[1.367182 1.367182 1.367182] [0. 0. 1.]
[-0.57735027 -0.57735027 -0.57735027] [ 0.  0. -1.]
[[ 9.24075272e-01 -4.59104470e-01 -2.85029212e-16]
 [-4.64970802e-01  9.24075272e-01  4.71845332e-16]
 [ 9.50729695e-18 -1.63723412e-17  1.00000000e+00]]
Congrates, we found an appx CSL!

U1 = 
[[ 1 -6]
 [-6  5]
 [ 5  1]]; sigma_1 = 54

U2 = 
[[-3 -5]
 [-8  3]
 [ 0  0]]; sigma_2 = 49

D = 
[[ 0.99590432  0.00730638 -0.0032107 ]
 [-0.0032107   0.99590432  0.00730638]
 [ 0.00730638 -0.0032107   0.99590432]]

axis = [-7.47674648 -7.47674648 -7.47674648] ; theta = 15.21



In [5]:
my_interface.compute_bicrystal_two_D(hkl_1 = hkl_1, hkl_2=hkl_2, \
                                     normal_ortho = True, plane_ortho = True, \
                                     lim = 50, tol_ortho = 1e-2)
my_interface.get_bicrystal(two_D = True, xyz_1 = [1,1,1], xyz_2 = [1,1,1])

[[ 13.67182    2.734364]
 [-16.406184  13.67182 ]
 [  2.734364 -16.406184]]
[[ 13.67182    2.734364]
 [-16.406184  13.67182 ]
 [  2.734364 -16.406184]]
[[ 9.47211475e+00 -7.41964412e-16  1.24257699e-15]
 [ 2.51391483e-16  2.15304037e+01 -1.07652018e+01]
 [-2.78325182e-16  9.20825833e-16  1.86458765e+01]]


## Generate LAMMPS commands to sampling system energy in the CNID

The interface structure can be complex, especially when we can have a 'triclinic' cell.

Therefore, we provide some functions here to help you write some necessary input commands for LAMMPS.

Note that it is far more effective to do looping by LAMMPS's looping variables than by generating many different atom files!

1. Define regions & groups

In [6]:
my_interface.CNID

array([[-2.734364,  2.734364],
       [ 2.734364,  0.      ],
       [ 0.      , -2.734364]])

In [ ]:
my_interface.define_lammps_regions(['right'],[5.2],[10.5])

You can see that here we generated a 'blockfile' file defining a
'right' group, which will be displaced during the simulation.

2. Define translation variables

To verify the periodicity of CNID, we translate in a 2x2 expansion of CNID divided into a fine 50x50 grids 
(dont worry, this won't take a long time :D)

In [8]:
v1, v2 = 2*dot(my_interface.orient, my_interface.CNID).T
write_trans_file(v1,v2,100,100)

Here we generated a 'paras' file, including the looping variables and 
the translation variables which will be applied in the 'displace_atoms' command.

Now, put these two files in the 'CNID_check' folder and run LAMMPS in this folder!